#import

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import f1_score
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from  sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from collections import Counter
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from keras.layers import Dense
from keras.models import Sequential
%matplotlib inline

#Google drive

In [2]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


#Data

In [3]:
names = ['aff', 'variation']

In [4]:
aff = pd.read_csv('/gdrive/My Drive/current_affiliations.tsv', sep='\t', index_col=0, names=names)

In [5]:
aff

,aff,variation
0,MPSU,Moscow Pedagogical State University
0,MPSU,"Moscow Pedagogical State University, Moscow, R..."
0,MPSU,MPSU
0,MPSU,"ИВ РАН, МПГУ, Москва, Россия"
0,MPSU,Московский государственный педагогический унив...
...,...,...
10000,Unknown,Хельсинки
10000,Unknown,Чехия
10000,Unknown,Днепропетровск
10000,Unknown,Нью-Йорк


In [6]:
values = aff.aff.value_counts()

In [7]:
rare_aff = aff[aff.aff.isin(values[values<=2].index)]  #  2 и менее вариаций у аффилиаций в датафрейме
double_rare_aff = pd.concat([rare_aff, rare_aff]) # удваиваем датафрейм, чтобы конкатенировать к основному датасету
aff = pd.concat([aff, double_rare_aff]) # теперь единичных вариаций минимум 3, а тех, где было только 2 вариации, стало 6

In [8]:
aff = pd.concat([aff, aff]) # также увеличим в 2 раза весь датасет, чтобы можно было сплитить в нужном соотношении и при этом были представлены все классы

In [9]:
new_ind = dict() # 
i = 0
for index in aff.index:
  if index not in new_ind.keys():
    new_ind[index] = i
    i += 1

old_ind = dict()
for key, value in new_ind.items():
  old_ind[value]=key

aff['new_ind'] = aff.index
aff['new_ind'] = aff.new_ind.map(new_ind)
aff

,aff,variation,new_ind
0,MPSU,Moscow Pedagogical State University,0
0,MPSU,"Moscow Pedagogical State University, Moscow, R...",0
0,MPSU,MPSU,0
0,MPSU,"ИВ РАН, МПГУ, Москва, Россия",0
0,MPSU,Московский государственный педагогический унив...,0
...,...,...,...
2543,"Murom Institute of Vladimir State University, ...",Муромский институт (филиал,379
2543,"Murom Institute of Vladimir State University, ...",Муромский институт,379
2544,"JINR Laboratory of High Energy Physics, Dubana...",Институт физики высоких энергий,380
2545,"Center of applied communication, Moscow, Russia",Центр прикладных коммуникаций,381


In [10]:
X = aff.variation
y = aff.new_ind

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, stratify=y_train, test_size=0.25, random_state=42)

In [12]:
len(y_train.unique()), len(y_test.unique()), len(y_val.unique()) # все классы представлены в каждом из датасетов

(385, 385, 385)

#Classifier logreg

In [149]:
vect = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=1000)
X_train_cv, X_test_cv  = vect.fit_transform(X_train), vect.transform(X_test)

In [150]:
%%time
reg_model = linear_model.LogisticRegression(max_iter=5000)
reg_model.fit(X_train_cv, y_train)

CPU times: user 9min 24s, sys: 6min 21s, total: 15min 46s
Wall time: 8min 35s


In [ ]:
y_pred = reg_model.predict(X_test_cv)
cl_rep = classification_report(y_test, y_pred, output_dict=True)

In [159]:
cl_rep['accuracy']

0.9730941704035875

In [160]:
cl_rep['weighted avg']

{'f1-score': 0.9712596259646796,
 'precision': 0.9761144031603675,
 'recall': 0.9730941704035875,
 'support': 892}

In [161]:
cl_rep['macro avg']

{'f1-score': 0.9723856083449645,
 'precision': 0.9742814977230562,
 'recall': 0.9782251837456335,
 'support': 892}

# NN dataset

In [13]:
vocab = Counter()

for af in aff.variation:
    vocab.update(af)

In [14]:
char2id = {'UNK':1, 'PAD':0}

for char in vocab:
    char2id[char] = len(char2id)

In [15]:
id2char = {i:char for char, i in char2id.items()}

In [16]:
X = []

for word in aff.variation:
    ids = np.array([char2id.get(token, 1) for token in word])
    X.append(ids)

In [84]:
MAX_LEN = max(len(x) for x in X)
MAX_LEN

232

In [87]:
def prepare_dataset(data, max=232):
  res = []
  for word in data:
    ids = np.array([char2id.get(token, 1) for token in word])
    res.append(ids)
  res = tf.keras.preprocessing.sequence.pad_sequences(res, maxlen=max)
  return res

In [88]:
X_train_rn = prepare_dataset(X_train)
X_val_rn = prepare_dataset(X_val)
X_test_rn = prepare_dataset(X_test)

y_train_rn = tf.keras.utils.to_categorical(y_train)
y_val_rn = tf.keras.utils.to_categorical(y_val)
y_test_rn = tf.keras.utils.to_categorical(y_test)

In [57]:
X_train_rn.shape, y_train.shape

((2673, 232), (2673,))

# CNN

In [143]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(char2id), 
                                    input_length=MAX_LEN, output_dim=10))
model.add(tf.keras.layers.Conv1D(kernel_size=8, filters=4, strides=1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(600, activation="relu"))
model.add(tf.keras.layers.Dense(385, activation='softmax'))
opt = SGD(lr=0.0005, momentum=0.9)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 232, 10)           1690      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 225, 4)            324       
_________________________________________________________________
flatten_8 (Flatten)          (None, 900)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 600)               540600    
_________________________________________________________________
dense_17 (Dense)             (None, 385)               231385    
Total params: 773,999
Trainable params: 773,999
Non-trainable params: 0
_________________________________________________________________


In [144]:
model.fit(X_train_rn, y_train_rn, 
          validation_data=(X_val_rn, y_val_rn),
          batch_size=1,
          epochs=25, 
          callbacks=[callback])

Epoch 1/25
2673/2673 [==============================] - 12s 4ms/step - loss: 5.7198 - accuracy: 0.0543 - val_loss: 5.4575 - val_accuracy: 0.0752
Epoch 2/25
2673/2673 [==============================] - 11s 4ms/step - loss: 5.4436 - accuracy: 0.0691 - val_loss: 5.2425 - val_accuracy: 0.0763
Epoch 3/25
2673/2673 [==============================] - 11s 4ms/step - loss: 5.2219 - accuracy: 0.0773 - val_loss: 4.9253 - val_accuracy: 0.0786
Epoch 4/25
2673/2673 [==============================] - 11s 4ms/step - loss: 4.8154 - accuracy: 0.0871 - val_loss: 4.2133 - val_accuracy: 0.1336
Epoch 5/25
2673/2673 [==============================] - 12s 4ms/step - loss: 3.6679 - accuracy: 0.2535 - val_loss: 2.6105 - val_accuracy: 0.4725
Epoch 6/25
2673/2673 [==============================] - 16s 6ms/step - loss: 1.9109 - accuracy: 0.5706 - val_loss: 1.7522 - val_accuracy: 0.6465
Epoch 7/25
2673/2673 [==============================] - 13s 5ms/step - loss: 0.9667 - accuracy: 0.7713 - val_loss: 1.4758 - val_ac

In [ ]:
y_pred_rn = model.predict_classes(X_test_rn)
cl_rep = classification_report(y_test, y_pred_rn, output_dict=True)

In [146]:
cl_rep['accuracy']

0.8396860986547086

In [147]:
cl_rep['weighted avg']

{'f1-score': 0.8323198895352768,
 'precision': 0.849427681114571,
 'recall': 0.8396860986547086,
 'support': 892}

In [148]:
cl_rep['macro avg']

{'f1-score': 0.8916653769971935,
 'precision': 0.8967536105351231,
 'recall': 0.9110761942602569,
 'support': 892}

# LSTM

In [128]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(char2id), 
                                    input_length=MAX_LEN, output_dim=10))
model.add(tf.keras.layers.LSTM(8, input_shape=(MAX_LEN, len(vocab))))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(600, activation="relu"))
model.add(tf.keras.layers.Dense(385, activation='softmax'))
opt = SGD(lr=0.01, momentum=0.9)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [129]:
model.fit(X_train_rn, y_train_rn, 
          validation_data=(X_val_rn, y_val_rn),
          batch_size=1,
          epochs=25,
          callbacks=[callback])

Epoch 1/25
2673/2673 [==============================] - 107s 39ms/step - loss: 5.6536 - accuracy: 0.0661 - val_loss: 5.3510 - val_accuracy: 0.0752
Epoch 2/25
2673/2673 [==============================] - 104s 39ms/step - loss: 5.3356 - accuracy: 0.0750 - val_loss: 5.1578 - val_accuracy: 0.0224
Epoch 3/25
2673/2673 [==============================] - 104s 39ms/step - loss: 5.1113 - accuracy: 0.0724 - val_loss: 4.9330 - val_accuracy: 0.0943
Epoch 4/25
2673/2673 [==============================] - 105s 39ms/step - loss: 4.8662 - accuracy: 0.0793 - val_loss: 4.5508 - val_accuracy: 0.1066
Epoch 5/25
2673/2673 [==============================] - 105s 39ms/step - loss: 4.5080 - accuracy: 0.0946 - val_loss: 4.4986 - val_accuracy: 0.1021
Epoch 6/25
2673/2673 [==============================] - 104s 39ms/step - loss: 4.1065 - accuracy: 0.1267 - val_loss: 4.2809 - val_accuracy: 0.0584
Epoch 7/25
2673/2673 [==============================] - 104s 39ms/step - loss: 3.8951 - accuracy: 0.1441 - val_loss: 3

In [ ]:
y_pred_rn = model.predict_classes(X_test_rn)
cl_rep = classification_report(y_test, y_pred_rn, output_dict=True)

In [131]:
cl_rep['accuracy']

0.21188340807174888

In [135]:
cl_rep['weighted avg']

{'f1-score': 0.16380753801660777,
 'precision': 0.17779992911852302,
 'recall': 0.21188340807174888,
 'support': 892}

In [136]:
cl_rep['macro avg']

{'f1-score': 0.15315795864172788,
 'precision': 0.14827972517418644,
 'recall': 0.203209103250829,
 'support': 892}